In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# 讀取車牌圖片
image_path = "2.jpeg"
image = cv2.imread(image_path)

# 灰階處理
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 二值化處理（使用 Otsu）
_, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# 形態學操作（閉運算以連接字符）
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
morph = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

# 輪廓偵測
contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 篩選與排序文字輪廓
char_regions = []
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    if 10 < w < 60 and 20 < h < 70:  # 這些範圍視車牌大小可能要調整
        char_regions.append((x, y, w, h))

# 依據x排序（從左到右）
char_regions = sorted(char_regions, key=lambda b: b[0])

# 從原圖切出字元圖像
char_images = [image[y:y+h, x:x+w] for (x, y, w, h) in char_regions]

# 顯示切割後的字元圖像
char_pil_images = [Image.fromarray(cv2.cvtColor(char, cv2.COLOR_BGR2RGB)) for char in char_images]
char_pil_images[:6]  # 最多先展示前6個字元



[]

In [2]:
import cv2
import numpy as np
import os

def segment_chars_by_projection(img_path, 
                                 out_dir="chars", 
                                 thresh_block=11, 
                                 thresh_C=2, 
                                 min_char_width=10):
    """
    對車牌圖做投影分割，輸出每張只含一個字元的小圖。
    參數:
      img_path: 來源車牌圖路徑
      out_dir: 切好的字元圖要存在這個資料夾
      thresh_block, thresh_C: adaptiveThreshold 參數
      min_char_width: 忽略過窄的區段
    """
    # 讀圖 & 預處理
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    bw = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY_INV, thresh_block, thresh_C)
    # 形態學去雜訊
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    bw = cv2.morphologyEx(bw, cv2.MORPH_OPEN, kernel, iterations=1)
    
    # 計算垂直投影：對每個 x 列，數白點(255)數量
    vertical_proj = np.sum(bw==255, axis=0)
    
    # 自動找「谷底」：即連續低於平均值的一段作為切分區間
    mean_val = np.mean(vertical_proj)
    is_low = vertical_proj < (mean_val*0.5)   # 低於一半平均視為間隔
    splits = []
    in_gap = False
    for x, flag in enumerate(is_low):
        if flag and not in_gap:
            # 進入低谷
            in_gap = True
            start = x
        elif not flag and in_gap:
            # 低谷結束，記錄中心點
            end = x-1
            splits.append((start+end)//2)
            in_gap = False
    # 若最後一段是低谷
    if in_gap:
        end = len(is_low)-1
        splits.append((start+end)//2)
    
    # 加上最前面、最後面邊界
    boundaries = [0] + splits + [bw.shape[1]]
    
    # 建立輸出資料夾
    os.makedirs(out_dir, exist_ok=True)
    
    # 依 boundaries 切圖
    char_paths = []
    for i in range(len(boundaries)-1):
        x1, x2 = boundaries[i], boundaries[i+1]
        w = x2 - x1
        if w < min_char_width: 
            continue  # 太窄就跳過
        crop = img[:, x1:x2]
        # 再次以垂直方向做水平投影修剪上下多餘空白
        gray_crop = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
        bw_crop = cv2.threshold(gray_crop,0,255,
                                cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]
        horiz_proj = np.sum(bw_crop==255, axis=1)
        ys = np.where(horiz_proj>0)[0]
        if ys.size:
            y1, y2 = ys[0], ys[-1]
            crop = crop[y1:y2+1, :]
        # 存檔
        out_path = os.path.join(out_dir, f"char_{i}.png")
        cv2.imwrite(out_path, crop)
        char_paths.append(out_path)
    
    print(f"共切出 {len(char_paths)} 張字元圖，保存在 '{out_dir}'")
    return char_paths

if __name__ == "__main__":
    # 範例呼叫
    chars = segment_chars_by_projection(
        img_path="2.jpeg",
        out_dir="chars",
        thresh_block=15,
        thresh_C=3,
        min_char_width=15
    )


共切出 1 張字元圖，保存在 'chars'
